In [ ]:
!pip install transformers

import re
import pandas as pd
import numpy as np

#importing all the pytorch packages
import torch
import tensorflow as tf
from transformers import pipeline, AutoTokenizer

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [1]:
# Get GPU device name
device_name = tf.test.gpu_device_name()

if device_name == '/device:GPU:0':
  print('Found GPU at: {}'.format(device_name))
else:
  raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [2]:
if torch.cuda.is_available():

  # tell Pytorch to use the GPU
  device = torch.device('cuda')

  print('There are %d GPU(s) available.' % torch.cuda.device_count())
  print('We will use the GPU:', torch.cuda.get_device_name(0))

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [32]:
train_path = '/content/gdrive/My Drive/Colab Notebooks/Data/Svensk_NER/train_corpus.tsv'
test_path = '/content/gdrive/My Drive/Colab Notebooks/Data/Svensk_NER/test_corpus.tsv'

df_train = pd.read_csv(train_path, engine="python",delimiter="\t",header=None,encoding='utf-8',error_bad_lines=False)
df_test = pd.read_csv(test_path, engine="python",delimiter="\t",header=None,encoding='utf-8',error_bad_lines=False)

Skipping line 38068: unexpected end of data


In [33]:
df_train.head(10)

,0,1
0,I,0
1,längden,0
2,torde,0
3,få,0
4,moderna,0
5,par,0
6,överleva,0
7,om,0
8,en,0
9,part,0


In [34]:
df_train.describe()

,0,1
count,53591,52604
unique,13204,6
top,.,0
freq,3185,49737


In [36]:
# display the unique tags for the corpus
df_train[1].unique()

array(['0', 'PER', 'ORG', 'LOC', 'MISC', 'PRG', None], dtype=object)

In [37]:
def load_ner_data(file_path):
    myoutput,words,tags = [],[],[]
    fh = open(file_path)
    for line in fh:
        line = line.strip()
        if "\t" not in line:
            #Sentence ended.
            myoutput.append([words,tags])
            words,tags = [],[]
        else:
            word, tag = line.split("\t")
            words.append(word)
            tags.append(tag)
    fh.close()
    return myoutput

In [38]:
ner_train = load_ner_data(train_path)
ner_test = load_ner_data(test_path)

In [39]:
#merge 
df = pd.merge(df_train,df_test)
label = list(df[1].values)#we will be using this to make a set of all unique labels

In [45]:
print('Train shape:',np.array(ner_train).shape)
print('Test shape:',np.array(ner_test).shape)

Train shape: (32762, 2)
Test shape: (25204, 2)


In [47]:
#lets convert them to dataframs for easier handling
df_train = pd.DataFrame(ner_train,columns=["sentence","labels"])
df_test = pd.DataFrame(ner_test,columns=["sentence","labels"])

In [54]:
sentences = list(df_train['sentence'])+list(df_test['sentence'])
print("No of sentences:",len(sentences))
labels = list(df_train['labels'])+list(df_test['labels']) 
print("No of labels:",len(labels))

No of sentences: 57966
No of labels: 57966


In [55]:
def untokenize(words):
  '''puts the text back together as sentences (based on CoNLL)'''
    text = ' '.join(words)
    step1 = text.replace("`` ", '"').replace(" ''", '"').replace('. . .',  '...')
    step2 = step1.replace(" ( ", " (").replace(" ) ", ") ")
    step3 = re.sub(r' ([.,:;?!%]+)([ \'"`])', r"\1\2", step2)
    step4 = re.sub(r' ([.,:;?!%]+)$', r"\1", step3)
    step5 = step4.replace(" '", "'").replace(" n't", "n't").replace(
         "can not", "cannot")
    step6 = step5.replace(" ` ", " '")
    return step6.strip()

In [56]:
sentences = [untokenize(sent) for sent in sentences]
sentences[1]

'Gunhild Westman, tidigare lektor i pedagogik vid Uppsala universitet och som forskat på barns lek, anser att naturlig lek är avgörande för barns utveckling.'

In [139]:
#BERT's implementation comes with a pretained tokenizer and a defined vocabulary
tokenizer = AutoTokenizer.from_pretrained('KB/bert-base-swedish-cased', do_lower_case=True, keep_accents=True)

#tokenizing the text 
tokenized_texts = list(map(lambda x: ['[CLS]'] + tokenizer.tokenize(x) + ['[SEP]'] , sentences))
print(tokenized_texts[0])


['[CLS]', 'i', 'lang', '##den', 'torde', 'fa', 'moderna', 'par', 'over', '##leva', 'om', 'en', 'part', 'bara', 'lever', 'i', 'den', 'lilla', '[UNK]', 'och', 'den', 'andra', 'i', 'den', 'stora', '.', '[SEP]']


In [150]:
text = "Den tysk-brittiska researrangören TUI överväger avyttringar av verksamhet eller en nyemission för att få bukt på skuldberget \
som byggts upp under covid-19-pandemin. Aktien faller drygt 6 procent på Londonbörsen efter beskedet. Beskedet lämnas av koncernchefen \
Fritz Joussen i samband med en rapport enligt vilken TUI gjorde en underliggande rörelseförlust på 1,1 miljarder euro under årets andra \
kvartal. Omsättningen under kvartalet rasade med 98,5 procent till 72 miljoner euro. Joussen vill inte spekulera i storleken på nyemissionen \
som övervägdes eller vilka delar av koncernen som kan bli aktuella att sälja. TUI har nyligen säkrat ett nytt likviditetslån från Tysklands \
regering, som hamnar ovanpå andra statliga lånepaket på sammanlagt 1,8 miljarder euro som TUI tagit emot sedan april. Enligt TUI kommer det \
dröja till 2022 innan resandet är tillbaka i någon form av nytt normaltillstånd. Bokningsläget i sommar ligger enligt Joussen ungefär 81 \
procent lägre i år jämfört med i fjol och läget är känsligt, inte minst sedan brittiska resor till Spanien åkt på ett bakslag till följd av \
nya brittiska karantänsregler."

In [151]:
nlp = pipeline('ner', model='KB/bert-base-swedish-cased-ner', tokenizer='KB/bert-base-swedish-cased-ner', ignore_labels=[])

l = []
t = nlp(text)
in_word=False

for i,token in enumerate(t):
    if token['entity'] == 'O':
        in_word = False
        continue

    if token['word'].startswith('##'):
        # deal with (one level of) orphaned ##-tokens
        if not in_word:
            l += [ t[i-1] ]
            l[-1]['entity'] = token['entity']
        
        l[-1]['word'] += token['word'][2:]
    else:
        l += [ token ]

    in_word = True

l

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity': 'ORG', 'index': 7, 'score': 0.9967266321182251, 'word': 'TUI'},
 {'entity': 'MSR', 'index': 39, 'score': 0.9998525381088257, 'word': 'drygt'},
 {'entity': 'MSR', 'index': 40, 'score': 0.9997631311416626, 'word': '6'},
 {'entity': 'MSR',
  'index': 41,
  'score': 0.9997367262840271,
  'word': 'procent'},
 {'entity': 'ORG',
  'index': 43,
  'score': 0.6026151180267334,
  'word': 'Londonbörsen'},
 {'entity': 'PER', 'index': 53, 'score': 0.9979046583175659, 'word': 'Fritz'},
 {'entity': 'PER',
  'index': 54,
  'score': 0.9978270530700684,
  'word': 'Joussen'},
 {'entity': 'ORG', 'index': 63, 'score': 0.99785315990448, 'word': 'TUI'},
 {'entity': 'MSR', 'index': 71, 'score': 0.9998416304588318, 'word': '1'},
 {'entity': 'MSR', 'index': 72, 'score': 0.9998123049736023, 'word': ','},
 {'entity': 'MSR', 'index': 73, 'score': 0.9998078346252441, 'word': '1'},
 {'entity': 'MSR',
  'index': 74,
  'score': 0.9998399615287781,
  'word': 'miljarder'},
 {'entity': 'MSR', 'index': 75, 'sco